In [1]:
from csv import reader,DictWriter
import re
import pandas as pd

In [2]:
import logging    # first of all import the module

logging.basicConfig(filename='std.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')
logging.warning('This message will get logged on to a file')

In [3]:
formatting_tags = ['h1','h2','h3','h4','h5','h6']

In [4]:
def return_tag_type(tag):
    bold_tags = ['b','strong']
    italic_tags = ['i','em']
    heading_tags = ['h1','h2','h3','h4','h5','h6']
    deletion_tags = ['del','s']
#     emphasis_tags = ['b','strong','i','em'] 
    if tag in bold_tags:
        return "bold"
    elif tag in italic_tags:
        return "italic"
#     if tag in emphasis_tags:
#         return "emphasis"
    elif tag in heading_tags:
        return "heading"
    elif tag in deletion_tags:
        return "delete"
    else:
        return tag

In [5]:
def remove_tags(text):
    Tag_first = re.compile(r'<[^\/>]+>')
    Tag_end = re.compile(r'<[^>]+>')
    without_first_tag = Tag_first.sub('',text)
    return Tag_end.sub(' ', without_first_tag)

In [6]:
result_dataframe = pd.read_csv('/home/shahla/ProjectWorkHTML/Posts_csv_files/WithoutCodeBlock/without_pre_htmls_last.csv')
# For getiing heading type of data only, for othe types, change the start and end tags as the expected tags and
# put the rest of the tags in the other tags
start_all_tags = ['<h1>','<h2>','<h3>','<h4>','<h5>','<h6>']
end_all_tags = ['</h1>','</h2>','</h3>','</h4>','</h5>','</h6>']
other_all_tags = ['<p>','</p>','<li>','</li>','<ul>','</ul>','<del>','<s>','<code>','</del>','</s>','</code>','<i>','<em>','<b>','<strong>','</i>','</em>','</b>','</strong>']

with open('/home/shahla/ProjectWorkHTML/Posts_csv_files/Emphasis_dataset/ver2_emphasis_only_last.csv', 'w', newline='',encoding="utf8") as csvfile:
    fieldnames = ['word','Tags']
    writer = DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for index, row in result_dataframe.iterrows():
        answer_html = str(row['Answers'])
        word_tag_dictionary = {}
        tag_name = ''
#         creating a space starting and ending of the tags
        s = answer_html.replace("<"," <").replace(">","> ").replace(",",", ")
        pattern = re.compile(r'(\.(\s))')
        sentence = re.sub(pattern, ' ', s)
        arr=sentence.split()
#         print(s)
        for tag in formatting_tags:
            reg_str = "<" + tag + "> (.*?) <\/" + tag + ">" 
           
            res = re.findall(reg_str, s,flags=re.DOTALL)

            word_count_arr = []
            for i in range(len(res)):
                word_count = len(res[i].split())
                word_count_arr.append(word_count)
            word_tag_dictionary[tag] = word_count_arr
            
# start_bool for checking whether it is the starting of the tag
# tag_count is to keep track of the tagged words

        start_bool={'code':False,'b':False,'strong':False,'i':False,'em':False,'h1':False,'h2':False,'h3':False,'h4':False,'h5':False,'h6':False,'del':False,'s':False}
        tag_count= {'code':-1,'b':-1,'strong':-1,'i':-1,'em':-1,'h1':-1,'h2':-1,'h3':-1,'h4':-1,'h5':-1,'h6':-1,'del':-1,'s':-1}
        tag_position= {'code':0,'b':0,'strong':0,'i':0,'em':0,'h1':0,'h2':0,'h3':0,'h4':0,'h5':0,'h6':0,'del':0,'s':0}
        
        
        any_format_tag = bool([a for a in word_tag_dictionary.values() if a != []])
        word_length_all={'code':0,'b':0,'strong':0,'i':0,'em':0,'h1':0,'h2':0,'h3':0,'h4':0,'h5':0,'h6':0,'del':0,'s':0}
        
        try:
            if any_format_tag == 0:
                continue
    #             for i in arr:
    #                 writer.writerow({fieldnames[0]: i,fieldnames[1]:'O'})
    #                 print('O')
            else:
                for i in arr:
                    if i in start_all_tags:
                        index = i.strip('</>')
                        start_bool[index] = True
                        tag_count[index] += 1
    #                     print(i)
    #                     print(word_tag_dictionary[index],tag_position[index],tag_count[index])
                        word_length_all[index] = word_tag_dictionary[index][tag_position[index]]
                        tag_name = index
    #                     print(index)
                    elif i in end_all_tags:
                        index = i.strip('</>')
                        start_bool[index] = False
                        tag_count[index] = -1
                        tag_position[index] += 1
    #                     print(i)
#         check if any other tag is true
                    elif bool([a for a in start_bool.values() if a == True]):
    #                     print(start_bool)
                        for key in start_bool.keys():
                            if start_bool[key] == True:
                                tag_type = return_tag_type(key)
    #                             print(word_length_all[key])
                                if tag_count[key] == 0:
    #                                 print('B-'+tag_type)
                                    ner_tag = 'B-'+tag_type

                                    writer.writerow({fieldnames[0]: i,fieldnames[1]:ner_tag})

                                elif tag_count[key] < word_length_all[key] -1:
    #                                 print('I-'+tag_type)
                                    ner_tag = 'I-'+tag_type
                                    writer.writerow({fieldnames[0]: i,fieldnames[1]:ner_tag})
                                elif tag_count[key] == word_length_all[key] -1:
                                    ner_tag = 'E-'+tag_type
                                    writer.writerow({fieldnames[0]: i,fieldnames[1]:ner_tag})
                                tag_count[key] += 1

    #                     print(i)
                    elif i in other_all_tags:
                        continue
                    else:
                        writer.writerow({fieldnames[0]: i,fieldnames[1]:'O'})
            #             print("Nothing")
#                 writer.writerow({fieldnames[0]: "(EOS)",fieldnames[1]:'O'})
    #             print(tag_count)
        except:
            continue

FileNotFoundError: [Errno 2] No such file or directory: '/home/shahla/ProjectWorkHTML/Posts_csv_files/WithoutCodeBlock/without_pre_htmls_last.csv'